# Image Classification Model Training

In [1]:
import os
import math
import tensorflow as tf

## Select Parameters for Training

In [2]:
epochs = 100
input_size = (224, 224) 
batch_size = 20 # Select max size for the GPU RAM
val_split = 0.15 # ratio for validation set

## Setup all Paths Needed for Training

In [3]:
data_root = "./dataset cv ai 2021" # Edit this to point to your dataset location

# make root directory for all training outputs
root_dir = "./training_outputs"
# Make folder for viewing augmented images 
augment_dir = "{}/augmented_images".format(root_dir)
# tensorboard logs
logs = "{}/tensorboard_logs".format(root_dir)
# for saving keras models
models_path = "{}/models".format(root_dir)

# automatically create the folders if they dont already exist
os.makedirs(root_dir, exist_ok=True)
os.makedirs(augment_dir, exist_ok=True)
os.makedirs(logs, exist_ok=True)
os.makedirs(models_path, exist_ok=True)

## Data Preperation

In [4]:
# Create datagenerator object for loading and preparing image data for training
dataflow_kwargs = dict(
    directory = data_root, 
    target_size = input_size, 
    class_mode = "categorical",
    batch_size = batch_size,
    shuffle = True,     
    interpolation = "bilinear",
    )

# add minor augmentation to help prevent overfitting
datagen_args_train = dict(
    rescale = 1./255,
    validation_split = val_split,
    # image augmentation params
    rotation_range = 8,
    width_shift_range = 0.05,
    height_shift_range = 0.1,
    brightness_range = (0.7, 1.0),
    shear_range = 2,
    zoom_range = [0.8, 1.0],
    vertical_flip = False,
    horizontal_flip = True,
    fill_mode = "constant",
        )
datagen_args_val = dict(
    rescale = 1./255,
    validation_split = val_split
        )

datagen_train = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_args_train)
datagen_val = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_args_val)

train_generator = datagen_train.flow_from_directory(
    subset = "training",
    # view augemented images
    save_to_dir = augment_dir,
    save_prefix = "train",
    save_format = "jpg",
    **dataflow_kwargs)

# Dont use augmentation on the validation set
valid_generator = datagen_val.flow_from_directory(
    subset = "validation",
    **dataflow_kwargs)



Found 255 images belonging to 3 classes.
Found 45 images belonging to 3 classes.


## Create callbacks for training

In [5]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = logs, 
    histogram_freq = 1,
    write_steps_per_second = True, # Only available in tf2.5
    update_freq = "epoch",
    embeddings_freq = 0 # freq for viewing embedding layers
    )
# Save the model when the validation loss improves
save_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = models_path + "/epoch-{epoch:04d}/model.h5", # Create new sub dir for each saved model
    monitor = "val_loss",
    verbose = 1, 
    save_best_only = True, 
    save_weights_only = False, # False: h5.pb, True: checkpoint file
    )

In [6]:
# Set steps so that all images are processed per epoch
steps_per_epoch = math.ceil(train_generator.samples / train_generator.batch_size)
validation_steps = math.ceil(valid_generator.samples / valid_generator.batch_size)

## Create Convolutional Neural Network

In [7]:
# Create very small model to prevent overfitting
num_classes = 3 # change later to dynamicaly get class number

keras_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(input_size[0], input_size[1], 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

## Compile the Model

In [8]:
keras_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), 
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
    metrics = ['accuracy'])

keras_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        3

## Train the Model

In [9]:
history = keras_model.fit(
    train_generator,
    epochs = epochs, 
    steps_per_epoch = steps_per_epoch,
    validation_data = valid_generator,
    validation_steps = validation_steps,
    callbacks = [tensorboard_callback, 
                save_callback]
                        )

Epoch 1/100


C:\Users\Christian.Orr.ENERSERV\Anaconda3\envs\TF2-6\lib\site-packages\keras\backend.py:4846: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


13/13 [==============================] - 10s 520ms/step - loss: 1.0998 - accuracy: 0.2941 - val_loss: 1.0982 - val_accuracy: 0.2444

Epoch 00001: val_loss improved from inf to 1.09825, saving model to ./training_outputs/models/epoch-0001\model.h5
Epoch 2/100
13/13 [==============================] - 6s 456ms/step - loss: 1.0977 - accuracy: 0.3922 - val_loss: 1.0961 - val_accuracy: 0.2889

Epoch 00002: val_loss improved from 1.09825 to 1.09609, saving model to ./training_outputs/models/epoch-0002\model.h5
Epoch 3/100
13/13 [==============================] - 6s 442ms/step - loss: 1.0934 - accuracy: 0.3765 - val_loss: 1.0921 - val_accuracy: 0.4444

Epoch 00003: val_loss improved from 1.09609 to 1.09212, saving model to ./training_outputs/models/epoch-0003\model.h5
Epoch 4/100
13/13 [==============================] - 7s 528ms/step - loss: 1.0970 - accuracy: 0.3490 - val_loss: 1.0939 - val_accuracy: 0.4889

Epoch 00004: val_loss did not improve from 1.09212
Epoch 5/100
13/13 [===============

# Load and test the best performing model

In [10]:
model_epoch = "0026" # Select the epoch to run testing on

path = "{}/epoch-{}/model.h5".format(models_path, model_epoch)
loaded_model = tf.keras.models.load_model(path)
print("Evaluating model")
reeval_loss, reeval_acc = loaded_model.evaluate(valid_generator, verbose=2)

OSError: SavedModel file does not exist at: ./training_outputs/models/epoch-0026/model.h5\{saved_model.pbtxt|saved_model.pb}